# Getting to know HCIPy

Docs: 
https://hcipy.readthedocs.io/en/latest/

What do I need?
- aperture
- coronagraph
- optical propagation


In [ ]:
from hcipy import *
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# Setting up pupil and focal grid and propagator
pupil_grid = make_pupil_grid(1024)
focal_grid = make_focal_grid(pupil_grid, 8, 32)    # 8 pixels perlambda/D, 32 lambda/D radius of total image
prop = FraunhoferPropagator(pupil_grid, focal_grid)   # this is without a coronagraph

## Apertures

### Circular aperture

In [ ]:
circ = circular_aperture(1)
circ = evaluate_supersampled(circ, pupil_grid, 8)

imshow_field(circ)
plt.show()

### Hexagonal aperture

In [ ]:
hexag = hexagonal_aperture(1)
hexag = evaluate_supersampled(hexag, pupil_grid, 8)

imshow_field(hexag)
plt.show()

### Magellan aperture

In [ ]:
magellan = make_magellan_aperture(1)
magellan = evaluate_supersampled(magellan, pupil_grid, 8)

imshow_field(magellan)
plt.show()

### Luvoir A aperture (old)

In [ ]:
luvoir = make_luvoir_a_aperture(1)
luvoir = evaluate_supersampled(luvoir, pupil_grid, 4)    # this takes quite a while

imshow_field(luvoir)
plt.show()

## Coronagraph

### Perfect coronagraph

#### Circular aperture

In [ ]:
# Create coronagraph propagator on aperture
coro_perf = PerfectCoronagraph(circ)

# Create a wavefront from aperture
wf = Wavefront(circ)

# Apply coronagraph propagator of wavefront to Lyot plane
lyot_plane = coro_perf(wf)

# Display
imshow_field(np.log10(lyot_plane.intensity), vmin=-8, vmax=0)
plt.colorbar()
plt.show()

In [ ]:
# Add Lyot stop and display image
lyot_stop = Apodizer(circular_aperture(0.99)(pupil_grid))

img = prop(lyot_stop(lyot_plane))
img_ref = prop(wf)

imshow_field(np.log10(img.intensity / img_ref.intensity.max()), vmin=-12)
plt.title("Circular aperture perfect coronagraph")
plt.colorbar()
plt.show()

In [ ]:
# Compare to propagation without coronagraph
imshow_field(np.log10(img_ref.intensity), vmin=-8, vmax=0)
plt.title("Circular aperture reference image")
plt.colorbar()
plt.show()

#### Luvoir aperture

In [ ]:
# Create a wavefront from aperture
wf = Wavefront(luvoir)

# Apply coronagraph propagator of wavefront to Lyot plane
lyot_plane = coro_perf(wf)

# Display
imshow_field(np.log10(lyot_plane.intensity), vmin=-8, vmax=0)
plt.colorbar()
plt.show()

In [ ]:
# Add Lyot stop and display image
lyot_stop = Apodizer(circular_aperture(0.99)(pupil_grid))   # circular Lyot stop on Luvoir aperture... not great but works

img = prop(lyot_stop(lyot_plane))
img_ref = prop(wf)

imshow_field(np.log10(img.intensity / img_ref.intensity.max()), vmin=-12)
plt.title("Luvoir aperture perfect coronagraph")
plt.colorbar()
plt.show()

In [ ]:
# Compare to propagation without coronagraph
imshow_field(np.log10(img_ref.intensity), vmin=-8, vmax=0)
plt.title("Luvoir iaperture reference image")
plt.colorbar()
plt.show()

### Lyot coronagraph

#### Circular aperture

In [ ]:
# Create a focal plane mask
fpm = Apodizer(circular_aperture(0.05)(pupil_grid))

# Create Lyot coronagraph on pupil grid with FPM
coro_lyot = LyotCoronagraph(pupil_grid, fpm)

# Create a wavefront from aperture
wf = Wavefront(circ)

# Apply coronagraph propagator of wavefront to Lyot plane
lyot_plane = coro_lyot(wf)

# Display
imshow_field(np.log10(lyot_plane.intensity), vmin=-8, vmax=0)
plt.colorbar()
plt.show()

## Generate ATLAST pupil

Check Luvoir API for this, should be easy since I don't need spiders or omitting certain segments.

## Get segment coordinates

Need segment coordinates to get non-redundant pairs. They are generated while generating the aperture, so should be easy to extract.

## Mini-segment

Figure out how to generate and propagate mini-segment, since that is needed for PASTIS. It's already there, but I will have to adjust how to deal with it.